In [ ]:
! pip install transformers tokenizers datasets

### Train the tokenizer

In [ ]:
# For the user's convenience `tokenizers` provides some very high-level classes encapsulating
# the overall pipeline for various well-known tokenization algorithm. 
# Everything described below can be replaced by the ByteLevelBPETokenizer class. 

from tokenizers import Tokenizer
from tokenizers.decoders import ByteLevel as ByteLevelDecoder
from tokenizers.models import BPE
from tokenizers.normalizers import Lowercase, NFKC, Sequence
from tokenizers.pre_tokenizers import ByteLevel

# First we create an empty Byte-Pair Encoding model (i.e. not trained model)
_tokenizer = Tokenizer(BPE())

# Our tokenizer also needs a pre-tokenizer responsible for converting the input to a ByteLevel representation.
_tokenizer.pre_tokenizer = ByteLevel()

# And finally, let's plug a decoder so we can recover from a tokenized input to the original one
_tokenizer.decoder = ByteLevelDecoder()


In [ ]:
from tokenizers.processors import TemplateProcessing

_tokenizer.post_processor = TemplateProcessing(
    single="<s> $A </s>",
    special_tokens=[
        ("<s>", 0),
        ("</s>", 1),
    ],
)


In [ ]:
from tokenizers.trainers import BpeTrainer

# We initialize our trainer, giving him the details about the vocabulary we want to generate
trainer = BpeTrainer(vocab_size=500, show_progress=True, initial_alphabet=ByteLevel.alphabet(), special_tokens=[
            "<s>",
            "</s>",
            "<pad>",
            "<unk>",
            "<mask>"
        ])


_tokenizer.train(files=["2021_07_20_tokenized_tunes.txt"], trainer=trainer)

In [ ]:
# saving the tokenized data in our specified folder 
import os
save_path = 'tokenized_data'

In [ ]:
_tokenizer.save(save_path)

In [ ]:
_tokenizer

In [ ]:
#_tokenizer.enable_truncation(max_length=512)

### Convert the tokenizer 

In [ ]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast(tokenizer_file=save_path)

In [ ]:
tokenizer.add_special_tokens({
  "eos_token": "</s>",
  "bos_token": "<s>",
  "unk_token": "<unk>",
  "pad_token": "<pad>",
  "mask_token": "<mask>"
})

### Load Dataset

In [ ]:
from datasets import load_dataset, GenerateMode

dataset = load_dataset('text', data_files='2021_07_20_tokenized_tunes.txt', download_mode = GenerateMode.FORCE_REDOWNLOAD)


In [ ]:
datasets = dataset['train'].train_test_split(0.2)

### Tokenize Dataset

In [ ]:
tokenized_dataset = datasets.map(lambda x: tokenizer(x['text'], truncation=True, max_length=512), num_proc=4, batched=True, remove_columns=["text"])

### Additional Dataset Preprocessing 

In [ ]:
strategy = 'line'

In [ ]:
strategy = 'window'

In [ ]:
block_size = 128
def preprocess_window(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
max_length = 512
def preprocess_line(examples):
    results = {
        k: v[:max_length] for k,v in examples.items()
    }
    return results

In [ ]:
if strategy == 'line': 
    preprocessed_data = tokenized_dataset.map(
        preprocess_line,
        num_proc=4,
        batched=True,
        batch_size=100
    )
elif strategy == 'window':
    preprocessed_data = tokenized_dataset.map(
        preprocess_window,
        num_proc=4,
        batched=True,
        batch_size=100
    )

In [ ]:
from transformers import DataCollatorWithPadding

class DataCollatorWithLabelPadding(DataCollatorWithPadding):
    def __call__(self, features):
        batch = super().__call__(features)
        batch['labels'] = batch['input_ids'].detach().clone()
        return batch

In [ ]:
if strategy == 'line':
    data_collator = DataCollatorWithLabelPadding(
        tokenizer=tokenizer,
        padding='longest',
        max_length=512,
        pad_to_multiple_of=4,
    )
elif strategy == 'window':
     data_collator = DataCollatorWithPadding(
        tokenizer=tokenizer,
        padding='longest',
        max_length=512,
        pad_to_multiple_of=4,
    )

### Configure Model

In [ ]:
# I chose the hyperparameters here to get an architecture that
# had roughly the same number of paramters as FolkRNN. But I changed a lot of 
# things and IDK what the trade offs are.
from transformers import AutoTokenizer, AutoModelForCausalLM, GPT2Config
config = GPT2Config(
    vocab_size = tokenizer.vocab_size,
    n_positions = 512, 
    n_embd = 384,
    n_ctx = 512,
    n_layer = 6,
    n_inner = 6,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id,
    unk_token_id=tokenizer.unk_token_id,
    mask_token_id=tokenizer.mask_token_id,
)
model = AutoModelForCausalLM.from_config(config)

In [ ]:
model.num_parameters()

### Configure Training

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir='./results',          # output directory
  num_train_epochs=10,              # total # of training epochs
  per_device_train_batch_size=16,  # batch size per device during training
  per_device_eval_batch_size=16,   # batch size for evaluation
  evaluation_strategy = "epoch",
   # directory for storing logs
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=preprocessed_data['train'],
    eval_dataset=preprocessed_data['test'],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

### Run Training

In [ ]:
trainer.train()

### Examine Output

In [ ]:
def generate(start_text = "a b"):
    # encoding the input text
    input_ids = tokenizer.encode(start_text, return_tensors='pt')
    # getting out output
    beam_output = model.generate(
      input_ids,
      do_sample=True,
      max_length = 128,
      top_k=50,
      temperature = 0.8,
      num_return_sequences=5,
      beams=5, 
    
    )
    return beam_output

In [ ]:
output = generate()
for i in range(0, 5):
    print(tokenizer.decode(output[i]))

### TODO / things to try:
#### Pre-processing:
    - possibly shift to the same scale 
#### Tokenization:
    - look at other archtitectures for domain specific special tokens
    - if we can't get good results, data process + train manually to use our own function 
#### Data-processing:
    - find a better way to create training labels
    - tune batch_size and block_size
#### Model
    - tune architecture parameters
    - try alternative architectures (ProphetNet?)
#### Training
    - tune training parameters
#### Generation
    - figure out what all of the parameters in generate do 
    - write our own generation code?
#### Evaluation
    - look into quality metrics from existing research
    - write some code to detect training data plagiarism (generation spitting back an input)
    - write some code to check for structural integrity ?
#### Misc
    - figure out a good workflow to train and share model versions 
    

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('checkpoint')
model = AutoModelForCausalLM.from_pretrained('checkpoint')

In [ ]:
tokenizer

In [ ]:
### retrain the model with 
### add the metadata

## figure out how to best generate n tunes / figure out the randomness


